END TO END DEEP LEARNING PROJECT USING SIMPLE RNN


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [3]:
## load imdb dataset
#max_features is like voc_size in embedding.ipynb
max_features = 10000
(X_train,y_train),(X_test,y_test)=imdb.load_data(num_words=max_features)
#Print the shape of the data
#data will be downloaded in cache memory in local system
print(f'Training data shape:{X_train.shape}, Training labels shape:{y_train.shape}')
print(f'Testing data shape: {X_test.shape}, Testing labels shape: {y_test.shape}')


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training data shape:(25000,), Training labels shape:(25000,)
Testing data shape: (25000,), Testing labels shape: (25000,)


In [5]:
X_train[0],y_train[0]

([1,
  14,
  22,
  16,
  43,
  530,
  973,
  1622,
  1385,
  65,
  458,
  4468,
  66,
  3941,
  4,
  173,
  36,
  256,
  5,
  25,
  100,
  43,
  838,
  112,
  50,
  670,
  2,
  9,
  35,
  480,
  284,
  5,
  150,
  4,
  172,
  112,
  167,
  2,
  336,
  385,
  39,
  4,
  172,
  4536,
  1111,
  17,
  546,
  38,
  13,
  447,
  4,
  192,
  50,
  16,
  6,
  147,
  2025,
  19,
  14,
  22,
  4,
  1920,
  4613,
  469,
  4,
  22,
  71,
  87,
  12,
  16,
  43,
  530,
  38,
  76,
  15,
  13,
  1247,
  4,
  22,
  17,
  515,
  17,
  12,
  16,
  626,
  18,
  2,
  5,
  62,
  386,
  12,
  8,
  316,
  8,
  106,
  5,
  4,
  2223,
  5244,
  16,
  480,
  66,
  3785,
  33,
  4,
  130,
  12,
  16,
  38,
  619,
  5,
  25,
  124,
  51,
  36,
  135,
  48,
  25,
  1415,
  33,
  6,
  22,
  12,
  215,
  28,
  77,
  52,
  5,
  14,
  407,
  16,
  82,
  2,
  8,
  4,
  107,
  117,
  5952,
  15,
  256,
  4,
  2,
  7,
  3766,
  5,
  723,
  36,
  71,
  43,
  530,
  476,
  26,
  400,
  317,
  46,
  7,
  4,
  2,
  1029,
  

In [6]:
## Inspect a sample review and its label
sample_review=X_train[0]
sample_label=y_train[0]
# decode real sentence
print(f"Sample review (as integers): {sample_review}")
print(f"Sample label: {sample_label}")

Sample review (as integers): [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
Sample label: 1


In [9]:
## Mapping of words index back to words(for understanding)
word_index=imdb.get_word_index()
#word_index
reverse_word_index = {value: key for key, value in word_index.items()}
reverse_word_index
#reverse key word index


{34701: 'fawn',
 52006: 'tsukino',
 52007: 'nunnery',
 16816: 'sonja',
 63951: 'vani',
 1408: 'woods',
 16115: 'spiders',
 2345: 'hanging',
 2289: 'woody',
 52008: 'trawling',
 52009: "hold's",
 11307: 'comically',
 40830: 'localized',
 30568: 'disobeying',
 52010: "'royale",
 40831: "harpo's",
 52011: 'canet',
 19313: 'aileen',
 52012: 'acurately',
 52013: "diplomat's",
 25242: 'rickman',
 6746: 'arranged',
 52014: 'rumbustious',
 52015: 'familiarness',
 52016: "spider'",
 68804: 'hahahah',
 52017: "wood'",
 40833: 'transvestism',
 34702: "hangin'",
 2338: 'bringing',
 40834: 'seamier',
 34703: 'wooded',
 52018: 'bravora',
 16817: 'grueling',
 1636: 'wooden',
 16818: 'wednesday',
 52019: "'prix",
 34704: 'altagracia',
 52020: 'circuitry',
 11585: 'crotch',
 57766: 'busybody',
 52021: "tart'n'tangy",
 14129: 'burgade',
 52023: 'thrace',
 11038: "tom's",
 52025: 'snuggles',
 29114: 'francesco',
 52027: 'complainers',
 52125: 'templarios',
 40835: '272',
 52028: '273',
 52130: 'zaniacs',

In [10]:
#from tensorflow directory
decoded_review = ' '.join([reverse_word_index.get(i-3,'?') for i in sample_review])
decoded_review
# will replace i-3 to ? if the index is not found

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [12]:
from tensorflow.keras.preprocessing import sequence
max_len = 500 #max_length of sentence
X_train=sequence.pad_sequences(X_train,maxlen=max_len) #pre padding
X_test=sequence.pad_sequences(X_test,maxlen=max_len)
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [14]:
X_train[0] #pre-padding is done

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [17]:
#Train simple RNN
model = Sequential()
model.add(Embedding(max_features,128,input_length=max_len)) #embedding layer
#for converting word to vector
model.add(SimpleRNN(128,activation='relu'))
model.add(Dense(1,activation="sigmoid")) #for multi class classification , replace sigmoid with softmax

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [18]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [26]:
#create an instance of EarlyStopiing Callback
from tensorflow.keras.callbacks import EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',patience=5, restore_best_weights=True)
earlystopping

In [27]:
##Train the model with early stopping
history=model.fit(
    X_train,y_train,epochs=10,batch_size=32,
    validation_split=0.2,
    callbacks=[earlystopping]

)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 146s 230ms/step - accuracy: 0.6016 - loss: 11468360704.0000 - val_accuracy: 0.7158 - val_loss: 0.5483
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 204s 233ms/step - accuracy: 0.7928 - loss: 0.4751 - val_accuracy: 0.7642 - val_loss: 0.4871
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 202s 234ms/step - accuracy: 0.8722 - loss: 0.3446 - val_accuracy: 0.7770 - val_loss: 0.4989
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 201s 232ms/step - accuracy: 0.9003 - loss: 0.2636 - val_accuracy: 0.7704 - val_loss: 0.5129
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 201s 231ms/step - accuracy: 0.9233 - loss: 0.2108 - val_accuracy: 0.7714 - val_loss: 0.5562
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 204s 234ms/step - accuracy: 0.9373 - loss: 0.1689 - val_accuracy: 0.7738 - val_loss: 0.6072
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 200s 231ms/step - accuracy: 0.9533 - loss: 0.1262 - val_accuracy: 0.7560 - val_loss: 0.6278


In [29]:
model.save("my_model.keras")


In [28]:
#save the model
model.save('simple_rnn_imdb.h5')

In [31]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

In [32]:
model=load_model("/content/simple_rnn_imdb.h5")
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (32, 500, 128)              │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (32, 128)                   │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (32, 1)                     │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [33]:
#get weights
model.get_weights()

[array([[ 0.07430832,  0.28634   , -0.24168435, ..., -0.11445045,
         -0.19185318,  0.26595724],
        [-0.06487945,  0.01223515, -0.01544867, ...,  0.03446618,
          0.0219152 ,  0.00133086],
        [-0.02202466,  0.04566173, -0.03375842, ...,  0.06364542,
         -0.04852491, -0.05001558],
        ...,
        [-0.03187129, -0.06080389,  0.02419106, ..., -0.02176085,
          0.00948139,  0.03601389],
        [-0.00273601, -0.01988815, -0.02352384, ...,  0.0029873 ,
         -0.00535459, -0.02161107],
        [-0.04600506, -0.01648577,  0.02072599, ..., -0.00582918,
          0.04353468,  0.00340791]], dtype=float32),
 array([[-0.0262224 , -0.13041463, -0.12952316, ...,  0.01958117,
          0.14356865, -0.03789063],
        [-0.08565079, -0.1689578 ,  0.15338922, ..., -0.08705547,
         -0.11439866, -0.13181856],
        [-0.11388255, -0.11247594,  0.03876285, ..., -0.09789259,
         -0.07486095, -0.02756897],
        ...,
        [-0.09140641, -0.06683711, -0.1

In [44]:
# step 2: Helper functions
# function to decode reviews
def decode_review(encoded_review):
  return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

  # function to preprocess user input
def preprocess_text(text):
  words = text.lower().split()
  encoded_review = [word_index.get(word, 2) + 3 for word in words]
  padded_review = sequence.pad_sequences([encoded_review], maxlen = 500)
  return padded_review

In [52]:
## prediction function
def predict_sentiment(review):
  preprocessed_input = preprocess_text(review)
  prediction=model.predict(preprocessed_input)
  sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
  return sentiment, prediction[0][0]




In [53]:
#step 4: user input and Prediction
#example review for prediction
example_review = "This movie was fantastic! The acting was great and the plot was thrilling."
sentiment, score = predict_sentiment(example_review)
print(f'Review:{example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Review:This movie was fantastic! The acting was great and the plot was thrilling.
Sentiment: Negative
Prediction Score: 0.0023104839492589235


In [55]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00


In [62]:
## streamlit app
import streamlit as st
st.title('IMDB Movie Review Sentiment Analysis')
st.write('Enter a movie review to classify it as positive or negative')

#user input
user_input = st.text_area('Movie Review')
if st.button('Classify'):
  preprocess_input=preprocess_text(user_input)
  ##make prediction
  prediction = model.predict(preprocess_input)
  sentiment='Positive' if prediction[0][0] > 0.5 else ' Negative'
  #display the result
  st.write(f'Sentiment: {sentiment}')
  st.write(f'Prediction Score: {prediction[0][0]}')
else:
  st.write('Please enter a movie review.')

2025-04-09 18:31:10.424 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-09 18:31:10.426 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-09 18:31:10.427 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-09 18:31:10.427 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-09 18:31:10.428 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-09 18:31:10.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-09 18:31:10.431 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-09 18:31:10.432 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [63]:
#display the result
st.write(f'Sentiment: {sentiment}')
st.write(f'Prediction Score: {prediction[0][0]}')

2025-04-09 18:31:11.897 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-09 18:31:11.899 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-09 18:31:11.900 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-09 18:31:11.901 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


NameError: name 'prediction' is not defined